In [1]:
import pandas as pd
import numpy as np
from time import time

from preprocess.DataLoader import execute_data_loader
from preprocess.XmlTransformer import xml_transformation

execute_data_loader()

matches_data = pd.read_csv('../data/match_details.csv')
players_data = pd.read_csv('../data/player_attributes.csv')

In [2]:
df_transformed = xml_transformation(matches_data)

In [3]:
df_transformed['date'] = pd.to_datetime(df_transformed['date'])
df_transformed.sort_values(by='date', inplace=True)

In [4]:
df_transformed[['season', 'result_match']] = df_transformed[['season', 'result_match']].astype('category')

In [5]:
start = time()

players = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

def get_player_overall_rating(player_id):
    return players_data.loc[players_data['player_api_id'].eq(player_id)].sort_values(by='date',ascending=False).iloc[0]['overall_rating']

def get_player_id_for_team(row, player, team_type):
    player_id = row[player]

    if not np.isnan(player_id):
        return player_id

    # If the player ID is NaN
    player_id = df_transformed.loc[df_transformed[f'{team_type}_team'].eq(row[f'{team_type}_team'])][player].value_counts().idxmax()
    df_transformed.loc[df_transformed.match_api_id == row.match_api_id, player] = int(player_id)
    
    return player_id

def calculate_player_stat(match_row):
    player_stats_dict = {}

    for player in players:
        team_type = 'home' if 'home' in player else 'away'
        player_id = get_player_id_for_team(match_row, player, team_type)

        overall_ranking = get_player_overall_rating(player_id)
        name = 'overall_rating_{}'.format(player)

        player_stats_dict[name] = int(overall_ranking)
        player_stats_dict['match_api_id'] = match_row.match_api_id

    return player_stats_dict

player_stats_dict = df_transformed.apply(lambda row: calculate_player_stat(row), axis=1)
new_player_stats = pd.json_normalize(player_stats_dict)
df_transformed = pd.merge(df_transformed, new_player_stats, how='left', on='match_api_id')
df_transformed = df_transformed.drop(players, axis=1)

end = time()
print("Calculate player stats in {:.3f} minutes".format((end - start) / 60))

Calculate player stats in 0.449 minutes


In [6]:
start = time()

def get_overall_player_strength(match_row):
    """
    Calculate the overall player strenght.
    """
    stats_home_players = match_row.filter(regex='overall_rating_home_player')
    stats_away_players = match_row.filter(regex='overall_rating_away_player')

    home_team_strength = round(stats_home_players.values.mean(), 2)
    away_team_strength = round(stats_away_players.values.mean(), 2)

    return home_team_strength, away_team_strength


df_transformed[['home_team_strength', 'away_team_strength']] = df_transformed.apply(lambda row: get_overall_player_strength(row), axis=1, result_type='expand')

cols_to_remove = df_transformed.filter(like='overall_rating').columns
df_without_overall_ratings = df_transformed.drop(columns=cols_to_remove)

end = time()
print("Calculate player stats in {:.3f} minutes".format((end - start) / 60))

Calculate player stats in 0.014 minutes


In [7]:
def get_scored_goals(row, team):
    return row['home_team_goal'] if row['home_team'] == team else row['away_team_goal']

def average_scored_goals(team, match_date, df, n):
    """
    Calculate the average goals scored by a team over the last n matches before a given date.
    """
    team_matches = df[((df['home_team'] == team) | (df['away_team'] == team)) &
                      (df['date'] < match_date)].sort_values(by='date', ascending=False).head(n)
    
    return team_matches.apply(lambda row: get_scored_goals(row, team), axis=1).mean()

def count_average_goals_from_last_n_matches(row, df, n=3):
    home_team = row['home_team']
    away_team = row['away_team']
    match_date = row['date']

    avg_goals_home = average_scored_goals(home_team, match_date, df, n)
    avg_goals_away = average_scored_goals(away_team, match_date, df, n)

    return avg_goals_home, avg_goals_away

# Usage
N = 12
df_transformed[['home_avg_goals_scored', 'away_avg_goals_scored']] = df_transformed.apply(
    lambda row: count_average_goals_from_last_n_matches(row, df_transformed, n=N), 
    axis=1, 
    result_type='expand'
)

end = time()
print("Calculate average goals from {} matches in {:.3f} minutes".format(N, (end - start) / 60))

Calculate average goals from 12 matches in 0.141 minutes


In [9]:
N = 10
start = time()

def get_conceded_goals(row, team):
    if row['home_team'] == team:
        return row['away_team_goal']
    elif row['away_team'] == team:
        return row['home_team_goal']
    return 0

def average_conceded_goals(team, match_date, n, df):
    """
    Calculate the average conceded goals of a team over the last n matches before a given date.
    """
    team_matches = df[((df['home_team'] == team) | (df['away_team'] == team)) &
                      (df['date'] < match_date)].sort_values(by='date', ascending=False).head(n)

    return team_matches.apply(lambda row: get_conceded_goals(row, team), axis=1).mean()

def count_average_conceded_goals_from_last_n_matches(row, df, n=3):
    home_team = row['home_team']
    away_team = row['away_team']
    match_date = row['date']

    avg_goals_home = average_conceded_goals(home_team, match_date, n, df)
    avg_goals_away = average_conceded_goals(away_team, match_date, n, df)

    return avg_goals_home, avg_goals_away

df_transformed[['home_avg_goals_conceded', 'away_avg_goals_conceded']] = df_transformed.apply(lambda row: count_average_conceded_goals_from_last_n_matches(row, df_transformed, n=N), axis=1, result_type='expand')

end = time()
print("Calculate average goals from {} matches in {:.3f} minutes".format(N, (end - start) / 60))

Calculate average goals from 10 matches in 0.132 minutes


In [10]:
start = time()
def count_previous_wins(df, team, match_date, season):
    """
    Count the number of wins of a team before a given match_date in a given season.
    """
    team_matches = df[((df['home_team'] == team) | (df['away_team'] == team)) &
                      (df['date'] < match_date) &
                      (df['season'] == season)]
    
    home_wins = (team_matches['home_team'] == team) & (team_matches['home_team_goal'] > team_matches['away_team_goal'])
    away_wins = (team_matches['away_team'] == team) & (team_matches['home_team_goal'] < team_matches['away_team_goal'])
    
    return home_wins.sum() + away_wins.sum()

def get_team_wins(row, df):
    """
    Get the number of wins for both the home and away teams for a given match row.
    """
    home_team = row['home_team']
    away_team = row['away_team']
    match_date = row['date']
    season = row['season']

    home_wins = count_previous_wins(df, home_team, match_date, season)
    away_wins = count_previous_wins(df, away_team, match_date, season)

    return home_wins, away_wins

df_transformed[['home_wins', 'away_wins']] = df_transformed.apply(lambda row: get_team_wins(row, df_transformed), axis=1, result_type='expand')

end = time()
df_transformed['wins_diff'] = ((df_transformed['home_wins'] * df_transformed['home_team_strength'] - df_transformed['away_wins'] * df_transformed['away_team_strength']) / 10).astype(int)

In [14]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 44 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   match_api_id                   3040 non-null   int64         
 1   season                         3040 non-null   category      
 2   stage                          3040 non-null   int64         
 3   date                           3040 non-null   datetime64[ns]
 4   away_team                      3040 non-null   int64         
 5   home_team                      3040 non-null   int64         
 6   home_team_goal                 3040 non-null   int64         
 7   away_team_goal                 3040 non-null   int64         
 8   result_match                   3040 non-null   category      
 9   away_shoton                    3040 non-null   int64         
 10  away_possession                3040 non-null   int64         
 11  home_possession  